In [1]:
import javalang
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split


import keras
from keras import layers
import tensorflow as tf
from keras.models import Sequential, Model, model_from_json
from keras.utils import Sequence
from keras.layers import concatenate, Input, LSTM, Dense, Masking, TimeDistributed, Embedding

from sklearn.metrics import classification_report, roc_auc_score, accuracy_score

Using TensorFlow backend.


In [2]:
Train_path = "../../../Dataset/AST/log4j/df_log4j_v10.csv"
Test_path = "../../../Dataset/AST/log4j/df_log4j_v11.csv"

In [20]:
Train_path = "../../../Dataset/AST/log4j/df_log4j_v11.csv"
Test_path = "../../../Dataset/AST/log4j/df_log4j_v12.csv"

In [21]:
df = pd.read_csv(Train_path)

In [22]:
df[:3]

,Unnamed: 0,metric_name,java_name,file,label
0,0,org.apache.log4j.xml.examples.XCategory,./log4j-v_1_1/src/java/org/apache/log4j/xml/ex...,/*\n * Copyright (C) The Apache Software Found...,1
1,1,org.apache.log4j.helpers.DateLayout,./log4j-v_1_1/src/java/org/apache/log4j/helper...,/*\n * Copyright (C) The Apache Software Found...,1
2,2,org.apache.log4j.net.test.SocketMin,./log4j-v_1_1/src/java/org/apache/log4j/net/te...,/*\n * Copyright (C) The Apache Software Found...,0


In [23]:
def build_dataset(name, file, label, limited):
    vocab = {}
    in_valid = []
    input_list = []
    label_list = []

    for i, x in enumerate(file):
        try:
            tree = javalang.parse.parse(x)
        except:
            in_valid.append(name[i])
            continue
        input_ = []
        for path, node in tree:
            node_type = type(node)
            flag =0
            for limit in limited:
                if limit in str(node_type):
                    flag = 1
                    break
            if not flag:continue

            if node_type not in vocab:
                vocab[node_type] = len(vocab)
            input_.append(node_type)
        input_list.append(input_)
        if label[i]>0:
            label_list.append(1)
        else:label_list.append(0)
        
    print(in_valid)
    return input_list, label_list, vocab

In [24]:
limited = ["Invocation", "Class", "Declaration", "Statement", "Clause"]
input_list, label_list, vocab = build_dataset(df.metric_name, df.file, df.label,limited)

['org.apache.log4j.PropertyConfigurator', 'org.apache.log4j.test.Finalize', 'org.apache.log4j.NDC', 'org.apache.log4j.Category']


In [25]:
length = [len(x) for x in input_list]
np.median(length), np.max(length), len(vocab)

(57.0, 422, 31)

# MLP

In [26]:
X_train, y_train = input_list, label_list

In [27]:
def preprocess(input_list, vocab):
    X = np.zeros((len(input_list), len(vocab)))
    for i, x in enumerate(input_list):
        for ele in x:
            X[i][vocab[ele]] +=1
    return X

In [28]:
#max_length = max([len(x) for x in input_list])
X_train_d = preprocess(X_train, vocab)

In [29]:
X_train_d

array([[1., 2., 4., ..., 0., 0., 0.],
       [1., 1., 9., ..., 0., 0., 0.],
       [1., 1., 2., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 1., 0., 0.],
       [1., 1., 4., ..., 0., 0., 0.],
       [1., 1., 8., ..., 0., 0., 0.]])

In [30]:
input_dim = X_train_d.shape[1]
nb_classes = 1
hidden_size = 32
drop_rate = 0.5

model = Sequential()
model.add(Dense(hidden_size, input_dim=input_dim))
model.add(layers.Activation('relu'))
model.add(Dense(nb_classes))
model.add(layers.Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam')

print("Training...")
model.fit(X_train_d, y_train, epochs=20, batch_size=10, validation_split=0, verbose=1)

Training...
Epoch 1/20
100/100 [==============================] - 0s 1ms/step - loss: 0.7819
Epoch 2/20
100/100 [==============================] - 0s 106us/step - loss: 0.6876
Epoch 3/20
100/100 [==============================] - 0s 137us/step - loss: 0.6519
Epoch 4/20
100/100 [==============================] - 0s 127us/step - loss: 0.6192
Epoch 5/20
100/100 [==============================] - 0s 125us/step - loss: 0.5987
Epoch 6/20
100/100 [==============================] - 0s 127us/step - loss: 0.5793
Epoch 7/20
100/100 [==============================] - 0s 115us/step - loss: 0.5605
Epoch 8/20
100/100 [==============================] - 0s 124us/step - loss: 0.5451
Epoch 9/20
100/100 [==============================] - 0s 125us/step - loss: 0.5255
Epoch 10/20
100/100 [==============================] - 0s 123us/step - loss: 0.5089
Epoch 11/20
100/100 [==============================] - 0s 122us/step - loss: 0.4958
Epoch 12/20
100/100 [==============================] - 0s 115us/step - loss

# Testing

In [31]:
df = pd.read_csv(Test_path)

In [32]:
def build_test_dataset(name, file, label, limited, vocab):
    in_valid = []
    vocab_missing = []
    input_list = []
    label_list = []

    for i, x in enumerate(file):
        try:
            tree = javalang.parse.parse(x)
        except:
            in_valid.append(name[i])
            continue
        input_ = []
        for path, node in tree:
            node_type = type(node)
            flag =0
            for limit in limited:
                if limit in str(node_type):
                    flag = 1
                    break
            if not flag:continue

            if node_type not in vocab:
                vocab_missing.append(node_type)
                continue
            input_.append(node_type)
        input_list.append(input_)
        if label[i]>0:
            label_list.append(1)
        else:label_list.append(0)
        
    print(in_valid)
    print(vocab_missing)
    return input_list, label_list

In [33]:
limited = ["Invocation", "Class", "Declaration", "Statement", "Clause"]
input_list, label_list = build_test_dataset(df.metric_name, df.file, df.label,limited, vocab)

['org.apache.log4j.Category', 'org.apache.log4j.test.Finalize', 'org.apache.log4j.NDC', 'org.apache.log4j.PropertyConfigurator', 'org.apache.log4j.jmx.LoggerDynamicMBean', 'org.apache.log4j.lf5.viewer.categoryexplorer.CategoryNodeEditor']
[]


In [34]:
X_test, y_test = input_list, label_list
X_test_d = preprocess(X_test, vocab)

In [35]:
print("Generating test predictions...")
y_pred = model.predict(X_test_d, verbose=0).reshape(-1)
y_pred_class = [round(x) for x in y_pred]

Generating test predictions...


In [36]:
print(classification_report(y_test, y_pred_class, target_names = ["Non-Defect", "Defect"]))

              precision    recall  f1-score   support

  Non-Defect       0.04      0.62      0.07         8
      Defect       0.95      0.30      0.46       180

    accuracy                           0.31       188
   macro avg       0.49      0.46      0.26       188
weighted avg       0.91      0.31      0.44       188



In [37]:
roc_auc_score(y_test, y_pred)

0.4784722222222222